In [32]:
import sys 
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages')
sys.path.insert(0, '')

In [ ]:
'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\lib', 
'D:\\Anaconda\\envs\\tensorflow_cpu', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages'

In [31]:
sys.path.clear()

In [33]:
sys.path

['',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages',
 'D:\\Anaconda\\envs\\tensorflow_cpu',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip']

In [232]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [35]:
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import datetime
import json
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval


In [36]:
# Load Original Data
df_train = pd.read_csv('../../data/beer/Train.csv')
df_valid = pd.read_csv('../../data/beer/Valid.csv')
df_test = pd.read_csv('../../data/beer/Test.csv')

In [38]:
keyphrases = pd.read_csv('../../data/yelp/KeyPhrases.csv')['Phrases'].tolist()

In [44]:
def to_sparse_matrix(df, num_user, num_item, user_col, item_col, rating_col):

    dok = df[[user_col, item_col, rating_col]].copy()
    dok = dok.values
    dok = dok[dok[:, 2] > 0]
    shape = [num_user, num_item]

    return sparse.csr_matrix((dok[:, 2].astype(np.float32), (dok[:, 0], dok[:, 1])), shape=shape)

def generate_sparse():
    num_users = df_train['UserIndex'].nunique()
    num_items = df_train['ItemIndex'].nunique()
    R_train = to_sparse_matrix(df_train, num_users, num_items, 'UserIndex', 'ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rtrain.npz', R_train)
    
    R_valid = to_sparse_matrix(df_valid, num_users, num_items, 'UserIndex','ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rvalid.npz', R_valid)
    
    R_test = to_sparse_matrix(df_test, num_users, num_items, 'UserIndex', 'ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rtest.npz', R_test)

In [47]:
rating_col = 'Binary'
generate_sparse()

In [49]:
# Load U-I Data 
rtrain = load_npz("../../data/beer/Rtrain.npz")
rvalid = load_npz("../../data/beer/Rvalid.npz")
rtest = load_npz("../../data/beer/Rtest.npz")

In [234]:
sys.path

['',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages',
 'D:\\Anaconda\\envs\\tensorflow_cpu',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip']

In [235]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
        vals.extend(np.array([1]*len(key_vector)))
    return csr_matrix((vals, (rows, cols)), shape=shape)

def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [51]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output



In [57]:
rtrain

<6370x3668 sparse matrix of type '<class 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [58]:
# Generate U-K 
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:33<00:00, 19242.25it/s]


## Original Res

In [236]:
similarity = normalize(train(rtrain))
user_item_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2701.35it/s]


In [237]:
user_item_res = evaluate(user_item_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4218.62it/s]


In [59]:
# k = 100
user_item_res

{'MAP@10': (0.045144826330695706, 0.002354497505081906),
 'MAP@15': (0.042265773714731864, 0.0019824373381772792),
 'MAP@20': (0.040129338606794795, 0.0017567965943763916),
 'MAP@5': (0.04956399929959726, 0.003133599222210933),
 'MAP@50': (0.032943936395371624, 0.0012118312514037895),
 'NDCG': (0.10093614993594359, 0.003179003332122599),
 'Precision@10': (0.03883733146559272, 0.0017160204191020419),
 'Precision@15': (0.035171890503706296, 0.0014211672707212524),
 'Precision@20': (0.032708807564349504, 0.0012423089296096815),
 'Precision@5': (0.044440553318157945, 0.002423972508292176),
 'Precision@50': (0.024941341271230957, 0.0008480003467919996),
 'R-Precision': (0.03997492992806294, 0.002317948843344031),
 'Recall@10': (0.06845154159235434, 0.0040585896263153525),
 'Recall@15': (0.09096277675804605, 0.004630203266326043),
 'Recall@20': (0.11247720176840045, 0.005101760514228912),
 'Recall@5': (0.040952551758442035, 0.0032340960406314727),
 'Recall@50': (0.19980217392786573, 0.006326

In [231]:
# k = 450
user_item_res

{'MAP@10': (0.04508650112703693, 0.0023953721196312986),
 'MAP@15': (0.04235786629500515, 0.002008745280886499),
 'MAP@20': (0.04037892464111684, 0.0017758143748504913),
 'MAP@5': (0.04958267670577248, 0.0032171719228623035),
 'MAP@50': (0.03349842902297764, 0.0012156249160740703),
 'NDCG': (0.1055558286865324, 0.003300682664669801),
 'Precision@10': (0.03885484153388199, 0.0016956609885519017),
 'Precision@15': (0.03573221268896282, 0.0014173559588368445),
 'Precision@20': (0.033549290842234285, 0.0012619992825901349),
 'Precision@5': (0.04377517072316582, 0.0024269161122634064),
 'Precision@50': (0.02543862721064612, 0.000847509622685325),
 'R-Precision': (0.041259814644960036, 0.002453576631494307),
 'Recall@10': (0.07301143636028665, 0.004238573282812461),
 'Recall@15': (0.09722378576031213, 0.004835876328696097),
 'Recall@20': (0.11915257488522873, 0.005277230322027469),
 'Recall@5': (0.04207622316112506, 0.0032932693755679185),
 'Recall@50': (0.21016528709525042, 0.00653006031485

In [238]:
# k = 450 + normalize
user_item_res

{'MAP@10': (0.04507482774817742, 0.0023948292773977673),
 'MAP@15': (0.042346014103149456, 0.002008373036997442),
 'MAP@20': (0.0403700676848506, 0.0017754967143902589),
 'MAP@5': (0.04958267670577248, 0.0032171719228623035),
 'MAP@50': (0.03349825936719119, 0.0012154894707161914),
 'NDCG': (0.1055896605488807, 0.003300984064520391),
 'Precision@10': (0.03885484153388198, 0.0016962836353977373),
 'Precision@15': (0.03573221268896282, 0.0014173559588368445),
 'Precision@20': (0.033549290842234285, 0.0012622084555109561),
 'Precision@5': (0.04377517072316582, 0.0024269161122634064),
 'Precision@50': (0.025452635265277537, 0.0008478539889811184),
 'R-Precision': (0.04126221328445171, 0.0024536290298659913),
 'Recall@10': (0.07300935182834745, 0.004238699096148311),
 'Recall@15': (0.09722378576031213, 0.004835876328696097),
 'Recall@20': (0.1191627890917308, 0.00527751575008427),
 'Recall@5': (0.04207622316112506, 0.0032932693755679185),
 'Recall@50': (0.2102435626973289, 0.006529800368505

In [115]:
# With similarity diagnal values set to 0
user_item_prediction_score = predict(rtrain, 100, test, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)
user_item_res = evaluate(user_item_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4909.26it/s]


NameError: name 'user_item_res2' is not defined

In [116]:
user_item_res

{'MAP@10': (0.044927201196243396, 0.0023405055457096638),
 'MAP@15': (0.04209462168926361, 0.001969742340258044),
 'MAP@20': (0.039992225689903314, 0.0017459659531029864),
 'MAP@5': (0.049373139555244264, 0.0031164938614256506),
 'MAP@50': (0.03276724419279445, 0.0012026552159499519),
 'NDCG': (0.10107630572080299, 0.003202357091908435),
 'Precision@10': (0.03860970057783225, 0.001708331996864007),
 'Precision@15': (0.035160217124846786, 0.0014148878710044373),
 'Precision@20': (0.03267378742777097, 0.0012475288598677123),
 'Precision@5': (0.04458063386447208, 0.002411935874452824),
 'Precision@50': (0.024906321134652427, 0.0008456291160631515),
 'R-Precision': (0.04081498375317024, 0.0024099351750396583),
 'Recall@10': (0.06853533972922739, 0.004072049507316102),
 'Recall@15': (0.09172129488813952, 0.004652525383450842),
 'Recall@20': (0.11268694933391117, 0.005113369923395411),
 'Recall@5': (0.04136211988453176, 0.0032415315263509283),
 'Recall@50': (0.1997124648831125, 0.00631678222

## Unlearned Res

In [187]:
similarity = train(U_K)
unlearned_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5001.47it/s]


In [188]:
unlearned_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

In [204]:
similarity = train(U_K)
unlearned_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)
unlearned_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4762.80it/s]


{'MAP@10': (0.033381652783683956, 0.00204745944991459),
 'MAP@15': (0.03135201769766644, 0.0017148691445157166),
 'MAP@20': (0.0298541871885246, 0.001509988462353567),
 'MAP@5': (0.03656277359481702, 0.002758406492295717),
 'MAP@50': (0.025227820735318138, 0.001018012642898509),
 'NDCG': (0.08171829377702637, 0.002959208265512183),
 'Precision@10': (0.02869900192610752, 0.0014592935796887052),
 'Precision@15': (0.026381836222494598, 0.0011998924678014592),
 'Precision@20': (0.02482052180003502, 0.001051947713306281),
 'Precision@5': (0.03253370688145684, 0.0020857093082908224),
 'Precision@50': (0.02025914901068114, 0.0007167136070223958),
 'R-Precision': (0.030383064859440045, 0.002176689772649127),
 'Recall@10': (0.05380651555392237, 0.003761411130781288),
 'Recall@15': (0.07078943972395199, 0.004195596238003561),
 'Recall@20': (0.08826154969949812, 0.004670019397229334),
 'Recall@5': (0.03159219339370157, 0.002941337954346157),
 'Recall@50': (0.1681931591315732, 0.00602771925593589)

## Normalized Unlearned

In [209]:
similarity = normalize(train(U_K))
unlearned_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)
unlearned_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5113.35it/s]


{'MAP@10': (0.033381652783683956, 0.00204745944991459),
 'MAP@15': (0.03135201769766644, 0.0017148691445157166),
 'MAP@20': (0.0298541871885246, 0.001509988462353567),
 'MAP@5': (0.03656277359481702, 0.002758406492295717),
 'MAP@50': (0.025227820735318138, 0.001018012642898509),
 'NDCG': (0.08171829377702637, 0.002959208265512183),
 'Precision@10': (0.02869900192610752, 0.0014592935796887052),
 'Precision@15': (0.026381836222494598, 0.0011998924678014592),
 'Precision@20': (0.02482052180003502, 0.001051947713306281),
 'Precision@5': (0.03253370688145684, 0.0020857093082908224),
 'Precision@50': (0.02025914901068114, 0.0007167136070223958),
 'R-Precision': (0.030383064859440045, 0.002176689772649127),
 'Recall@10': (0.05380651555392237, 0.003761411130781288),
 'Recall@15': (0.07078943972395199, 0.004195596238003561),
 'Recall@20': (0.08826154969949812, 0.004670019397229334),
 'Recall@5': (0.03159219339370157, 0.002941337954346157),
 'Recall@50': (0.1681931591315732, 0.00602771925593589)

# Learned Res

## Linear Regression

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize

In [63]:
X = U_K.todense()
y = train(rtrain)
reg = LinearRegression().fit(X, y)
lr_similarity = reg.predict(np.array(X))

In [64]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4989.88it/s]


In [91]:
lr_res

{'MAP@10': (0.028648396439063573, 0.0018400469256905094),
 'MAP@15': (0.0283316787948201, 0.001610900127524552),
 'MAP@20': (0.027879414774385672, 0.0014640831488321334),
 'MAP@5': (0.027961244382186418, 0.0023105314443931753),
 'MAP@50': (0.02513810297828301, 0.0010694045520145154),
 'NDCG': (0.07459171903871083, 0.0024768326099763667),
 'Precision@10': (0.028628961652950445, 0.0015334945590851372),
 'Precision@15': (0.027070565575205742, 0.0012964698511657843),
 'Precision@20': (0.02614253195587463, 0.0011665539834334715),
 'Precision@5': (0.029521975135703028, 0.002010099175833927),
 'Precision@50': (0.021190684643670108, 0.000793429184680451),
 'R-Precision': (0.025192652986106085, 0.001597767014265539),
 'Recall@10': (0.04585864252862637, 0.0033469994955957682),
 'Recall@15': (0.06492564899852431, 0.003971149216248076),
 'Recall@20': (0.081593969566691, 0.0043644956643599485),
 'Recall@5': (0.022789760560507657, 0.0023629644885964736),
 'Recall@50': (0.16346962972606943, 0.0058627

In [127]:
## With diag set to 0
X = U_K.todense()
# y = train(rtrain)
y = U_I_similarity
reg = LinearRegression().fit(X, y)
lr_similarity = reg.predict(np.array(X))

In [128]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 200, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5009.32it/s]


In [129]:
lr_res

{'MAP@10': (0.02863306818087067, 0.0018333400281224615),
 'MAP@15': (0.028232323621047134, 0.001599947921573674),
 'MAP@20': (0.02779711968023943, 0.0014531483467015924),
 'MAP@5': (0.02785326562773595, 0.002320384077959574),
 'MAP@50': (0.02510852743083808, 0.001067049005237428),
 'NDCG': (0.07417360933095597, 0.002478185906575529),
 'Precision@10': (0.028576431448082647, 0.0015101504496593471),
 'Precision@15': (0.02707056557520575, 0.001296107794743386),
 'Precision@20': (0.02612502188758536, 0.00115702044641536),
 'Precision@5': (0.028996673087025044, 0.001994688417539286),
 'Precision@50': (0.02121870075293294, 0.0008018237825573145),
 'R-Precision': (0.025050916631414236, 0.0015956024486881082),
 'Recall@10': (0.045664839020787835, 0.0032957843614861565),
 'Recall@15': (0.06436363032241992, 0.0038993935577644506),
 'Recall@20': (0.0814860175564146, 0.004366675444176269),
 'Recall@5': (0.022505595732506226, 0.0023570623901051323),
 'Recall@50': (0.1612765724307561, 0.0058166740582

## Ridge

In [177]:
from sklearn.linear_model import Ridge
X = U_K.todense()
y = train(rtrain)
# y = U_I_similarity
clf = Ridge(alpha=0.001).fit(X, y)

In [178]:
lr_similarity = clf.predict(np.array(X))

In [194]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5040.97it/s]


In [195]:
lr_res

{'MAP@10': (0.02863306818087067, 0.0018333400281224615),
 'MAP@15': (0.028232323621047134, 0.001599947921573674),
 'MAP@20': (0.02779711968023943, 0.0014531483467015924),
 'MAP@5': (0.02785326562773595, 0.002320384077959574),
 'MAP@50': (0.02510852743083808, 0.001067049005237428),
 'NDCG': (0.07417360933095597, 0.002478185906575529),
 'Precision@10': (0.028576431448082647, 0.0015101504496593471),
 'Precision@15': (0.02707056557520575, 0.001296107794743386),
 'Precision@20': (0.02612502188758536, 0.00115702044641536),
 'Precision@5': (0.028996673087025044, 0.001994688417539286),
 'Precision@50': (0.02121870075293294, 0.0008018237825573145),
 'R-Precision': (0.025050916631414236, 0.0015956024486881082),
 'Recall@10': (0.045664839020787835, 0.0032957843614861565),
 'Recall@15': (0.06436363032241992, 0.0038993935577644506),
 'Recall@20': (0.0814860175564146, 0.004366675444176269),
 'Recall@5': (0.022505595732506226, 0.0023570623901051323),
 'Recall@50': (0.1612765724307561, 0.0058166740582

In [191]:
unlearned_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

In [197]:
def tuning_ridge():
    res = []
    for k in [50,70,100,150,200,250,300,350,400,450,500,550,600]:
        lr_prediction_score = predict(rtrain, k, similarity, item_similarity_en= False)
        lr_predict = prediction(lr_prediction_score, 50, rtrain)
        lr_res = evaluate(lr_predict, rvalid)
        res.append(lr_res)
    return res

In [198]:
ridge_tuning_res = tuning_ridge()

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5589.19it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5133.71it/s]


In [203]:
for i in range(len(ridge_tuning_res)):
    print (i, ridge_tuning_res[i]['NDCG'])

0 (0.072528084174872, 0.002410374920819861)
1 (0.07378205696944092, 0.002459875766105016)
2 (0.07417360933095597, 0.002478185906575529)
3 (0.07569068174240462, 0.002521145340687763)
4 (0.07662225612464846, 0.0026304183240553674)
5 (0.07761031489604031, 0.0027082810478383608)
6 (0.07850295265917212, 0.0027475757572904868)
7 (0.07908263136428392, 0.0027689283833889507)
8 (0.07879029810698521, 0.0027525465447888605)
9 (0.0794821391006547, 0.0028267683170260342)
10 (0.0793745062876197, 0.0028006813707403444)
11 (0.07919224245876565, 0.0027935423511071347)
12 (0.07898878257151257, 0.0027665782842939096)


## Normalized ridge

In [205]:
X = normalize(U_K.todense())
y = normalize(train(rtrain))
# y = U_I_similarity
clf = Ridge(alpha=0.001).fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int32 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [206]:
lr_similarity = clf.predict(np.array(X))

In [207]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5009.26it/s]


In [208]:
lr_res

{'MAP@10': (0.03263396591929248, 0.0019468804416455736),
 'MAP@15': (0.031638375470453915, 0.0016638498848754906),
 'MAP@20': (0.030682716465145466, 0.001488148270339107),
 'MAP@5': (0.033189750773361344, 0.002531569958011714),
 'MAP@50': (0.0266903984580071, 0.0010506598328740783),
 'NDCG': (0.08478552305704484, 0.0028885505765459223),
 'Precision@10': (0.031132901418315533, 0.0015264339278548051),
 'Precision@15': (0.0287398587521158, 0.001270006219663063),
 'Precision@20': (0.027341971633689368, 0.0011270103589157922),
 'Precision@5': (0.033514270705655755, 0.0021126259276785905),
 'Precision@50': (0.021610926282612502, 0.0007567863313728811),
 'R-Precision': (0.03139425667660605, 0.0021073355445058567),
 'Recall@10': (0.05636449421586743, 0.003757381223990521),
 'Recall@15': (0.07740990068476872, 0.004399420210626781),
 'Recall@20': (0.09693583023402128, 0.004895937315332809),
 'Recall@5': (0.030727850842699442, 0.002795101908721248),
 'Recall@50': (0.1810198848801792, 0.0062849692

## One Layer Res

In [66]:
rtrain

<6370x3668 sparse matrix of type '<class 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [65]:
U_K

<6370x75 sparse matrix of type '<class 'numpy.int32'>'
	with 324742 stored elements in Compressed Sparse Row format>

In [76]:
U_K.todense().astype(float)

matrix([[ 7., 35., 27., ...,  2.,  2.,  0.],
        [16., 35., 23., ...,  2.,  0.,  5.],
        [ 0., 12.,  8., ...,  4.,  0.,  1.],
        ...,
        [ 6., 11.,  8., ...,  0.,  0.,  0.],
        [ 4., 17.,  5., ...,  0.,  0.,  0.],
        [10., 10.,  4., ...,  0.,  0.,  0.]])

In [210]:
R_train_keyphrase_tensor = tf.convert_to_tensor(normalize(U_K.todense().astype(float)))

In [211]:
R_train_keyphrase_tensor = tf.cast(R_train_keyphrase_tensor, tf.float32)

In [212]:
R_train_keyphrase_tensor

<tf.Tensor: id=210578, shape=(6370, 75), dtype=float32, numpy=
array([[0.07405438, 0.37027192, 0.28563833, ..., 0.0211584 , 0.0211584 ,
        0.        ],
       [0.15975258, 0.34945875, 0.22964433, ..., 0.01996907, 0.        ,
        0.04992268],
       [0.        , 0.3815778 , 0.2543852 , ..., 0.1271926 , 0.        ,
        0.03179815],
       ...,
       [0.31578946, 0.57894737, 0.42105263, ..., 0.        , 0.        ,
        0.        ],
       [0.1062988 , 0.4517699 , 0.1328735 , ..., 0.        , 0.        ,
        0.        ],
       [0.25967458, 0.25967458, 0.10386983, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [213]:
# R_Y_tensor = tf.convert_to_tensor(train(rtrain))
U_I_similarity = normalize(train(rtrain))
# np.fill_diagonal(U_I_similarity,0)
R_Y_tensor = tf.convert_to_tensor(U_I_similarity)

In [214]:
R_Y_tensor

<tf.Tensor: id=210579, shape=(6370, 6370), dtype=float32, numpy=
array([[0.31015164, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.26469022, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.40715232, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.32150823, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33610332,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.304243  ]], dtype=float32)>

In [215]:
train_dataset = [(R_train_keyphrase_tensor, R_Y_tensor)]

In [225]:
hidden_size = 100
# hidden_size2 = 1000
num_item_latent = 6370
initializer = tf.initializers.VarianceScaling(scale=2.0)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(hidden_size,
                                activation="relu",
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
# model.add(tf.keras.layers.Dense(hidden_size2,
#                                 activation="softmax",
#                                 kernel_initializer=initializer,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(tf.keras.layers.Dense(num_item_latent,
                                activation=None,
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

learning_rate = 1e-2
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.losses.MeanSquaredError()

# @tf.function
# def model_static(R_train_keyphrase_tensor, R_Y_tensor):
#     R_Y_pred_tensor = model(R_train_keyphrase_tensor)
#     loss = loss_fn(R_Y_tensor, R_Y_pred_tensor) + sum(model.losses)

num_epochs = 50
for epoch in range(num_epochs):
    # Iterate over the batches of a dataset.
    for x_batch_train, y_batch_train in train_dataset:
        with tf.GradientTape() as tape:
            y_batch_predict = model(x_batch_train)
            # Loss value for this minibatch
            loss_value = loss_fn(y_batch_train, y_batch_predict)
            # Add extra losses created during this forward pass:
            loss_value += sum(model.losses)
            print("Epoch: {}".format(epoch))
            print("L2 loss is: {}".format(sum(model.losses)))
            print("Sum loss is: {} \n".format(loss_value))

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

Epoch: 0
L2 loss is: 129.5386962890625
Sum loss is: 129.56866455078125 

Epoch: 1
L2 loss is: 115.17894744873047
Sum loss is: 115.19681549072266 

Epoch: 2
L2 loss is: 102.06536865234375
Sum loss is: 102.07555389404297 

Epoch: 3
L2 loss is: 90.15873718261719
Sum loss is: 90.1643295288086 

Epoch: 4
L2 loss is: 79.39019012451172
Sum loss is: 79.3932876586914 

Epoch: 5
L2 loss is: 69.6876449584961
Sum loss is: 69.6895751953125 

Epoch: 6
L2 loss is: 60.982879638671875
Sum loss is: 60.98441696166992 

Epoch: 7
L2 loss is: 53.20631408691406
Sum loss is: 53.20785903930664 

Epoch: 8
L2 loss is: 46.285301208496094
Sum loss is: 46.28702163696289 

Epoch: 9
L2 loss is: 40.147850036621094
Sum loss is: 40.1497802734375 

Epoch: 10
L2 loss is: 34.726280212402344
Sum loss is: 34.72837448120117 

Epoch: 11
L2 loss is: 29.956636428833008
Sum loss is: 29.95882797241211 

Epoch: 12
L2 loss is: 25.777036666870117
Sum loss is: 25.779251098632812 

Epoch: 13
L2 loss is: 22.127647399902344
Sum loss is: 

In [226]:
onelayer_similarity = model(R_train_keyphrase_tensor).numpy()

In [227]:
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5023.65it/s]


In [219]:
# One layer with 50 hidden 100 iteration loss 0.03
onelayer_res

{'MAP@10': (0.02503557601176232, 0.0017625984514546562),
 'MAP@15': (0.02427200431753049, 0.0014965028124616583),
 'MAP@20': (0.023542942134936472, 0.0013367478043353662),
 'MAP@5': (0.026229498628377982, 0.0023330042601491237),
 'MAP@50': (0.02085671981333879, 0.0009396802044936343),
 'NDCG': (0.06458872949738413, 0.00247671716591317),
 'Precision@10': (0.023568551917352482, 0.0013274805435194799),
 'Precision@15': (0.022015992529037534, 0.0011230171561493086),
 'Precision@20': (0.020985816844685694, 0.0009929836341171035),
 'Precision@5': (0.025109437926807917, 0.0018416765387969734),
 'Precision@50': (0.01772369112239538, 0.0006948991137942933),
 'R-Precision': (0.02245741837571335, 0.0017028854048965216),
 'Recall@10': (0.040569177114471666, 0.0032465116229396093),
 'Recall@15': (0.05462544083100035, 0.0036807779247994793),
 'Recall@20': (0.06733024874599443, 0.00402748919236708),
 'Recall@5': (0.02267004674235617, 0.0025181069989601537),
 'Recall@50': (0.13854845237773672, 0.00551

In [224]:
# One layer with 100 hidden 100 iteration loss 0.03
onelayer_res

{'MAP@10': (0.024942425227839342, 0.001753599015102256),
 'MAP@15': (0.024229579832801683, 0.0014915719612718153),
 'MAP@20': (0.02352988972177005, 0.0013332099664371537),
 'MAP@5': (0.026108679157182044, 0.0023184204478332634),
 'MAP@50': (0.020857001447091638, 0.0009389623189020825),
 'NDCG': (0.06467503313992916, 0.002474370689867957),
 'Precision@10': (0.02355104184906321, 0.0013304488397337573),
 'Precision@15': (0.022086032802194594, 0.0011192560306701274),
 'Precision@20': (0.020985816844685697, 0.000995373528265872),
 'Precision@5': (0.02507441779022938, 0.001836222348234274),
 'Precision@50': (0.01779022938189459, 0.000693842523292859),
 'R-Precision': (0.022246267028263624, 0.0016957257584338027),
 'Recall@10': (0.04050552128294041, 0.003242561893670821),
 'Recall@15': (0.055376677397139, 0.003709429051252763),
 'Recall@20': (0.06827447836960962, 0.004086801188359538),
 'Recall@5': (0.022682588139254365, 0.002518252387818311),
 'Recall@50': (0.13865705680982565, 0.00549899485

In [228]:
# One layer with 100 hidden 50 iteration loss 0.4
onelayer_res

{'MAP@10': (0.024567939064962355, 0.0017126678103723274),
 'MAP@15': (0.023573573161211055, 0.0014528270728242753),
 'MAP@20': (0.02284868231644502, 0.001290925635622992),
 'MAP@5': (0.025241347107920382, 0.002248852892312696),
 'MAP@50': (0.02062495837180942, 0.0008985522680279224),
 'NDCG': (0.06604758141356297, 0.002416536122886455),
 'Precision@10': (0.02313080021012082, 0.0013004267660010722),
 'Precision@15': (0.020895348158524486, 0.001061528037880263),
 'Precision@20': (0.020434249693573804, 0.0009390352905327568),
 'Precision@5': (0.024969357380493787, 0.0018197592567479309),
 'Precision@50': (0.01783225354578883, 0.0006710223785133447),
 'R-Precision': (0.02161900605361607, 0.001665857285943586),
 'Recall@10': (0.04149973853251357, 0.003327109700326088),
 'Recall@15': (0.05457084904471499, 0.0037409715295870236),
 'Recall@20': (0.06949475009117022, 0.004112532271125076),
 'Recall@5': (0.022011534947960643, 0.002416138348185266),
 'Recall@50': (0.1461468298260892, 0.0056253260

In [101]:
# one layer with 50 hidden size 1000 iteration loss 0.03
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4945.74it/s]


{'MAP@10': (0.021213114207335886, 0.0015441247684382993),
 'MAP@15': (0.021385818601717746, 0.0013445744371049868),
 'MAP@20': (0.02120335184705455, 0.0012152896832169232),
 'MAP@5': (0.0197040798459114, 0.001927988703831584),
 'MAP@50': (0.01940122728371883, 0.0008835459331518419),
 'NDCG': (0.058966962484347026, 0.0021559942015701335),
 'Precision@10': (0.022535457888285766, 0.0013075234772306158),
 'Precision@15': (0.021315589797466877, 0.0010918678163688046),
 'Precision@20': (0.02025914901068114, 0.0009645911685747211),
 'Precision@5': (0.02283312904920329, 0.00178173559032657),
 'Precision@50': (0.01703029241814043, 0.0006780414144150939),
 'R-Precision': (0.018570384041319415, 0.0013217106702485793),
 'Recall@10': (0.03737663702395449, 0.0030866689903839783),
 'Recall@15': (0.05330934947740096, 0.003634029695829435),
 'Recall@20': (0.06718245649487496, 0.004043861874344541),
 'Recall@5': (0.017695395613806054, 0.002061474609958878),
 'Recall@50': (0.1326869310614646, 0.005408477

In [105]:
# two layers with 1000,500 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5005.57it/s]


{'MAP@10': (0.021367001303527304, 0.0015465756923228947),
 'MAP@15': (0.021473255792114135, 0.0013488235935652229),
 'MAP@20': (0.021238032254585572, 0.0012210044232758212),
 'MAP@5': (0.019814393276133775, 0.0019325750334235165),
 'MAP@50': (0.01941099870491301, 0.0008885365739399716),
 'NDCG': (0.059086422460514335, 0.002171153165354127),
 'Precision@10': (0.022220276659078973, 0.0013091603340751815),
 'Precision@15': (0.021245549524309813, 0.001099541562289181),
 'Precision@20': (0.020127823498511645, 0.0009644702030258515),
 'Precision@5': (0.02307827000525302, 0.0017786428134943574),
 'Precision@50': (0.017163368937138854, 0.0006822963786275714),
 'R-Precision': (0.01854720485265285, 0.0013457366474314636),
 'Recall@10': (0.03636667166269102, 0.003022186541671131),
 'Recall@15': (0.05198258402777817, 0.0035617406148507782),
 'Recall@20': (0.06538572383034717, 0.00397471991358988),
 'Recall@5': (0.018754512582407862, 0.002204187988334541),
 'Recall@50': (0.13293780575424197, 0.0054

### Set diagnal value of U-U similarity to 0 and then learn 

In [126]:
# two layers with 1000,1000 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4974.13it/s]


{'MAP@10': (0.021155115580347588, 0.0015360366206437165),
 'MAP@15': (0.021489732697051907, 0.0013434719219333243),
 'MAP@20': (0.021325879552625313, 0.0012195828703010503),
 'MAP@5': (0.019381310920445923, 0.0019161506658330646),
 'MAP@50': (0.019459487724907486, 0.0008892249991592676),
 'NDCG': (0.05895307989821457, 0.0021539635794413457),
 'Precision@10': (0.022395377341971636, 0.0013130016333348547),
 'Precision@15': (0.021677464542111717, 0.0011023503734601241),
 'Precision@20': (0.020232883908247245, 0.000968337472236033),
 'Precision@5': (0.02269304850288916, 0.001763758560462977),
 'Precision@50': (0.017156364909823148, 0.0006828357556058593),
 'R-Precision': (0.018123183549739706, 0.0013209382625933882),
 'Recall@10': (0.03676556194755701, 0.003042629436270779),
 'Recall@15': (0.053657087597538844, 0.0036313985565362785),
 'Recall@20': (0.06572098019789895, 0.0039671821696352985),
 'Recall@5': (0.01752585067238741, 0.002077031751031006),
 'Recall@50': (0.13287713270957807, 0.0

In [130]:
# two layers with 1000,1000 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 200, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5041.27it/s]


{'MAP@10': (0.021765202491432573, 0.0015526845600560475),
 'MAP@15': (0.022049880410062516, 0.0013644357179226061),
 'MAP@20': (0.02180937910360531, 0.0012398563754525923),
 'MAP@5': (0.019955057491390885, 0.001933702044604114),
 'MAP@50': (0.019876764071423293, 0.0009036541747648264),
 'NDCG': (0.06005523877382802, 0.002181415237180367),
 'Precision@10': (0.023761162668534404, 0.0013408543906885092),
 'Precision@15': (0.021817545088425844, 0.0011176082906995454),
 'Precision@20': (0.02068814568376817, 0.0009855566555685455),
 'Precision@5': (0.02283312904920329, 0.0017578687007232115),
 'Precision@50': (0.01730695149711084, 0.0006824305773343914),
 'R-Precision': (0.018760004297206704, 0.0013097699165870852),
 'Recall@10': (0.04030599394664136, 0.0032072455844968126),
 'Recall@15': (0.052539438473823155, 0.0035653125479179902),
 'Recall@20': (0.0673341789140699, 0.004055092489593483),
 'Recall@5': (0.017723613895222083, 0.002101243798342838),
 'Recall@50': (0.13495388413124607, 0.0054

## Tune one-layer

In [233]:
def train_onelayer(num_epochs = 50, hidden_size = 100, num_item_latent = 6370):

    initializer = tf.initializers.VarianceScaling(scale=2.0)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(hidden_size,
                                    activation="relu",
                                    kernel_initializer=initializer,
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
    model.add(tf.keras.layers.Dense(num_item_latent,
                                    activation=None,
                                    kernel_initializer=initializer,
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

    learning_rate = 1e-2
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.losses.MeanSquaredError()

    for epoch in range(num_epochs):
        # Iterate over the batches of a dataset.
        for x_batch_train, y_batch_train in train_dataset:
            with tf.GradientTape() as tape:
                y_batch_predict = model(x_batch_train)
                # Loss value for this minibatch
                loss_value = loss_fn(y_batch_train, y_batch_predict)
                # Add extra losses created during this forward pass:
                loss_value += sum(model.losses)
                print("Epoch: {}".format(epoch))
                print("L2 loss is: {}".format(sum(model.losses)))
                print("Sum loss is: {} \n".format(loss_value))

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

# PLREC

In [140]:
from sklearn.utils.extmath import randomized_svd
def plrec(matrix_train, iteration=4, lamb=80, rank=200, seed=1, **unused):
    """
    Function used to achieve generalized projected lrec w/o item-attribute embedding
    :param matrix_train: user-item matrix with shape m*n
    :param iteration: number of power iterations in randomized svd
    :param lamb: parameter of penalty
    :param rank: latent dimension size
    :param seed: the seed of the pseudo random number generator to use when shuffling the data
    :return: prediction in sparse matrix
    """
    start_time = time.time()
    P, sigma, Qt = randomized_svd(matrix_train,
                                  n_components=rank,
                                  n_iter=iteration,
                                  random_state=seed)

    RQ = matrix_train.dot(sparse.csc_matrix(Qt.T*np.sqrt(sigma)))


    start_time = time.time()
    pre_inv = RQ.T.dot(RQ) + lamb * sparse.identity(rank, dtype=np.float32)
    inverse = sparse.linalg.inv(pre_inv.tocsc())
    Y = inverse.dot(RQ.T).dot(matrix_train)

    return np.array(RQ.todense()), np.array(Y.todense()), None

In [142]:
Y, RQt, Bias = plrec(rtrain, iteration=4, lamb=80, rank=200, seed=1)